# Import bibliotek

In [ ]:
import sys
import os
import csv
import time
import urllib3
import itertools
import numpy as np
import seaborn as sn
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_hub as hub
from PIL import Image
from matplotlib.backends.backend_pdf import PdfPages
from tensorflow.keras.preprocessing import image

print("TF version:", tf.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

# Przygotowanie danych do trenowania

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive",)
%cd /content/gdrive/MyDrive/ml_project/OIDv4_ToolKit

In [ ]:
def get_coco_labels():
    http = urllib3.PoolManager()
    r = http.request('GET', 'https://raw.githubusercontent.com/amikelive/coco-labels/master/coco-labels-paper.txt')
    dane = r.data.decode('UTF-8')
    my_data = dane.split('\n')
    # coco = {1: my_data[0], 2:my_data[12], 3:my_data[76]}
    coco = {i+1: my_data[i] for i in range(0,len(my_data))}
    coco.pop(91)
    return coco    

def get_downloadable_labels_str(coco):
    d = {value:1 for value in coco.values()}

    # Google's Open Images V4 dataset    Microsoft's COCO dataset
    d["traffic sign"] = d.pop("street sign")
    d["bull"] = d.pop("cow")
    d["glasses"] = d.pop("eye glasses")
    d["boot"] = d.pop("shoe")
    d["hair dryer"] = d.pop("hair drier")
    d["doughnut"] = d.pop("donut")
    d["microwave oven"] = d.pop("microwave")
    d["telephone"] = d.pop("cell phone")
    d["computer keyboard"] = d.pop("keyboard")
    d["remote control"] = d.pop("remote")
    d["computer mouse"] = d.pop("mouse")
    d["television"] = d.pop("tv")
    d["Kitchen dining room table"] = d.pop("dining table")
    d["plant"] = d.pop("potted plant")
    d["ski"] = d.pop("skis")
    d["volleyball"] = d.pop("sports ball")
    d["coffee cup"] = d.pop("cup")
    d["flying disc"] = d.pop("frisbee")

    labels = list(d.keys())
    lb = [x.capitalize() for x in labels]
    lb1 = [label.replace(" ","_") for label in lb]
    str1 = " "
    names = str1.join(lb1)
    print(names)
    return names

def get_paths_to_photos_and_labels(dataset_path = "OID/Dataset/train"):
    directory_contents = os.listdir(dataset_path)
    class_path_dict = {}
    for subdirectory in directory_contents:
        images = os.listdir(dataset_path + "/" + subdirectory)
        for image_ in images:
            if image_.endswith(".jpg"):
                full_image_path = dataset_path + "/" + subdirectory + "/" + image_
                if subdirectory.lower() in class_path_dict:
                    class_path_dict[subdirectory.lower()].append(full_image_path)
                else:
                    class_path_dict[subdirectory.lower()] = [full_image_path]
    return class_path_dict

def get_all_images_metadata_dict(path= "OID/csv_folder/train-annotations-bbox.csv"):
    my_dict = {}
    correct_class_id = class_id_to_coco_label_map.keys()
    with open(path, mode='r') as infile:
        reader = csv.reader(infile)
        for row in reader:
            if row[0] not in my_dict:
                if row[2] in correct_class_id:
                    my_dict[row[0]] = []
                    my_dict[row[0]].append(row[1:])
            else:
                if row[2] in correct_class_id:
                    my_dict[row[0]].append(row[1:])
    return my_dict

def get_class_id_to_open_image_label_map(path = "OID/csv_folder/class-descriptions-boxable.csv"):
    f = open(path)
    class_map = {}
    for line in f:
        data_line = line.rstrip().split(',')
        class_map[data_line[0]] = data_line[1]
    return class_map

def convert_open_image_label_to_coco(class_map):
    class_map = {value.lower():key for key, value in class_map.items()}

    class_map["street sign"] = class_map.pop("traffic sign")
    class_map["cow"] = class_map.pop("bull")
    class_map["eye glasses"] = class_map.pop("glasses")
    class_map["shoe"] = class_map.pop("boot")
    class_map["hair drier"] = class_map.pop("hair dryer")
    class_map["donut"] = class_map.pop("doughnut")
    class_map["microwave"] = class_map.pop("microwave oven")
    class_map["cell phone"] = class_map.pop("telephone")
    class_map["keyboard"] = class_map.pop("computer keyboard")
    class_map["remote"] = class_map.pop("remote control")
    class_map["mouse"] = class_map.pop("computer mouse")
    class_map["tv"] = class_map.pop("television")
    class_map["dining table"] = class_map.pop("kitchen & dining room table")
    class_map["potted plant"] = class_map.pop("plant")
    class_map["skis"] = class_map.pop("ski")
    class_map["sports ball"] = class_map.pop("volleyball")
    class_map["cup"] = class_map.pop("coffee cup")
    class_map["frisbee"] = class_map.pop("flying disc")

    return {value:key for key, value in class_map.items()}

def remove_non_coco_labels(class_id_to_coco_label_map):
    coco_list  = list(coco.values())
    class_id_to_only_coco_map = {}
    for key,val in class_id_to_coco_label_map.items():
        if val in coco_list:
            class_id_to_only_coco_map[key] = val
    return class_id_to_only_coco_map

def get_image_labels_with_bbox(metadata):
    labes_bboxes = []
    for entry in metadata:
        bbox = [float(i) for i in entry[3:7]]
        labes_bboxes.append([class_id_to_coco_label_map[entry[1]],bbox])
    return labes_bboxes

In [ ]:
coco = get_coco_labels()
coco_readable_to_index = {value:key for key, value in coco.items()}
get_downloadable_labels_str(coco)
# clone repository in the terminal, run it locally using commands with returned string from method "get_downloadable_labels_str(coco)" and upload to appropriate path in Google Drive("ml_project/OIDv4_ToolKit/OID/Dataset")
# git clone https://github.com/EscVM/OIDv4_ToolKit.git
# cd OIDv4_ToolKit/
# !pip3 install -r requirements.txt
# python main.py downloader --classes <paste_here_your_string> --type_csv <validation/test/train> --limit XX

In [ ]:
# obtain all needed dicts
class_id_to_open_image_label_map = get_class_id_to_open_image_label_map()
class_id_to_coco_label_map =  convert_open_image_label_to_coco(class_id_to_open_image_label_map)
class_id_to_coco_label_map = remove_non_coco_labels(class_id_to_coco_label_map)
all_images_metadata_dict = get_all_images_metadata_dict()


# Przetwarzanie odpowiednich danych i parametrów

In [ ]:
def predict(my_image, width=None, height=None):
    x = image.img_to_array(my_image)
    if width is not None and height is not None:
        x = tf.image.resize(x, [width, height], preserve_aspect_ratio=True)
    x = np.expand_dims(x, axis=0)
    start_time = time.time()
    preds = detector(x)
    end_time = time.time()
    inference_time = end_time-start_time
    # print("Inference time: ", inference_time)
    return preds, inference_time

def predict_single_image(path, width=None, height=None):
    image_name = os.path.basename(path).split(".")[0]
    image_ = Image.open(path)
    result, inference_time = predict(image_,width,height)
    results = {key:value.numpy() for key,value in result.items()}
    return results, image_name, inference_time

def compute_iou(groundtruth_box, detection_box):
    g_xmin, g_xmax, g_ymin, g_ymax = groundtruth_box
    d_ymin, d_xmin, d_ymax, d_xmax = detection_box

    xa = max(g_xmin, d_xmin)
    ya = max(g_ymin, d_ymin)
    xb = min(g_xmax, d_xmax)
    yb = min(g_ymax, d_ymax)

    intersection = max(0, xb - xa + 1) * max(0, yb - ya + 1)

    boxAArea = (g_xmax - g_xmin + 1) * (g_ymax - g_ymin + 1)
    boxBArea = (d_xmax - d_xmin + 1) * (d_ymax - d_ymin + 1)

    return intersection / float(boxAArea + boxBArea - intersection)

def update_confusion_matrix(oryginal_label_bbox, detected_label_bbox):
    if detected_label_bbox == None:
        confusion_matrix[coco_readable_to_index[oryginal_label_bbox[0]]-1][-1] += 1
    else:
        confusion_matrix[coco_readable_to_index[oryginal_label_bbox[0]]-1][coco_readable_to_index[detected_label_bbox[0]]-1] += 1

def perform_analysis(model_result, image_name):
    image_metadata_list = all_images_metadata_dict[image_name]
    labels_bboxes_original = get_image_labels_with_bbox(image_metadata_list)

    detected_classes = model_result['detection_classes'][0]
    detected_classes_accuracy = model_result['detection_scores'][0]
    detected_classes_bboxes = model_result['detection_boxes'][0]

    labels_bboxes_detected = []
    for d_class, score, bbox in zip(detected_classes,detected_classes_accuracy,detected_classes_bboxes):
        if score >= 0.5:
            labels_bboxes_detected.append([coco[d_class], list(bbox)])
    for bbox_oryginal in labels_bboxes_original:
        tmp = []
        best_detected_bbox = None
        for bbox_detected in labels_bboxes_detected:
            iou = compute_iou(bbox_oryginal[1], bbox_detected[1])
            if iou >= 0.5:
                tmp.append([iou,bbox_detected])
        if len(tmp) > 0:
            tmp.sort(key = lambda x: x[0])
            best_detected_bbox = tmp[-1][1]
        update_confusion_matrix(bbox_oryginal, best_detected_bbox)

def run_for_given_directory(dir_name):
    global confusion_matrix;
    confusion_matrix = [[0]*91 for i in range(90)]
    dataset_path = "OID/Dataset/train"
    subdirectory = dir_name
    images = os.listdir(dataset_path + "/" + subdirectory)
    val_err = 0
    print("subdirectory: " + subdirectory)
    for image_ in images:
        if image_.endswith(".jpg"):
            print("processing image: " + str(image_))
            full_image_path = dataset_path + "/" + subdirectory + "/" + image_
            try:
                results, image_name = predict_single_image(full_image_path)
            except ValueError:
                print("Val error for picture: ", image_)
                val_err += 1
                continue
            try:
                perform_analysis(results, image_name)
            except KeyError:
                print("Key error for picture: ", image_)
                continue

def get_all_detections(class_name):
    return confusion_matrix[coco_readable_to_index[class_name]-1][:-1]

def get_TP(class_name):
    return confusion_matrix[coco_readable_to_index[class_name]-1][coco_readable_to_index[class_name]-1]

def get_FN(class_name):
    return confusion_matrix[coco_readable_to_index[class_name]-1][-1]

def get_FP(class_name):
    return sum(get_all_detections(class_name)) - get_TP(class_name)

def get_precision(class_name):
    try:
        return get_TP(class_name) / get_all_detections(class_name)
    except ZeroDivisionError:
        return 0    

def get_recall(class_name):
    try:
        return get_TP(class_name) / ((get_TP(class_name))+get_FN(class_name))
    except ZeroDivisionError:
        return 0     

def get_F1_score(class_name):
    numerator = 2 * get_TP(class_name)
    denominator = numerator + get_FP(class_name) + get_FN(class_name)
    try:
        return numerator/denominator
    except ZeroDivisionError:
        return 0

# Skuteczność poszczególnej klasy i modelu dla konkretnej rozdzielczości

In [ ]:
def draw_bar_chart(values, labels, title, filename):
    _values, _labels = zip(*sorted(zip(values, labels)))
    positions = np.arange(len(_values)) + 0.5
    plt.figure(num=None, figsize=(27, 10))
    plt.barh(positions, _values, align='center')
    plt.yticks(positions, _labels)
    plt.tick_params(axis='y', labelsize=30)
    for index, v in enumerate(_values):
        plt.text(v, index, "{:.2f}".format(v), fontsize=30)
    plt.xlabel('')
    plt.title(title, fontsize=30)
    ax = plt.axes()
    ax.xaxis.set_tick_params(labelsize=30)
    ax.xaxis.grid(True)
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    plt.savefig(filename, papertype='a2')

def get_time_for_resolutions(times_with_res , class_name):
    time_with_res = []
    for resolution in times_with_res:
        print(resolution)
        time_with_res.append((resolution[0], resolution[1], resolution[2][class_name]))
    return time_with_res

def read_time(model_name):
    matrices = []
    files = os.listdir(model_name)
    for file in files:
        if 'inference_time' in file:
            resolution = file.split("_")[0]
            if resolution.split("x")[0] == "640":
                resolution_int = 0
                resolution = "Original"
            else:
                resolution_int = int(resolution.split("x")[0]) * int(resolution.split("x")[1])
            matrices.append((resolution_int, resolution, open_time(model_name + "/" + file)))
    matrices.sort(key = lambda x: x[0])
    return matrices

def open_time(filename):
    with open(filename, 'r') as f:
        l = eval(f.read())
    return l

import itertools

def draw_time(times_with_resolutions, name, model_names):
    fig = plt.figure()
    plt.rcParams["figure.figsize"] = (20,7)
    model_names_cycle = itertools.cycle(model_names)
    for time_with_resolution in times_with_resolutions:
        time_with_resolution.pop(0)
        x_positions = np.arange(len(time_with_resolution))
        values = [i[2] for i in time_with_resolution]
        labels = [i[1] for i in time_with_resolution]
        plt.plot(x_positions, values, linestyle='dashed', linewidth = 3,
        marker='o', markersize=12, label = next(model_names_cycle))
    plt.xticks(x_positions, labels)
    plt.legend(loc='best', prop={'size': 20})
    plt.tick_params(axis="both", labelsize=18)
    plt.title(name, fontsize=20)
    plt.xlabel("resolution", fontsize=20)
    plt.ylabel("time", fontsize=20)
    return fig

def print_pdf_for_model_time(model_names):
    times_with_res = []
    for model_name in model_names:
        times_with_res.append(read_time(model_name))
    if len(model_names) > 1:
        time_pp = PdfPages("inference_time_by_resolution_comparison.pdf")
    else:
        time_pp = PdfPages(model_names[0] + "_inference_time_by_resolution.pdf")
    time = []
    for class_name in coco.values():
        for time_with_res in times_with_res:
            time.append(get_time_for_resolutions(time_with_res, class_name))
        time_pp.savefig(draw_time(time, class_name, model_names))
        time.clear()
    time_pp.close()

def draw_normalized_TP_graph(model_name, width, height):
    tmp = []
    for row in confusion_matrix:
        normalized_row = []
        for value in row:
            normalized_row.append(value/(sum(row)))
        tmp.append(normalized_row)
    diag = [ tmp[i][i] for i in range(len(tmp)) ]
    diag = diag[:-1]
    coco_labels = list(get_coco_labels().values())
    print()
    title = "Normalized TP"
    folder = str(model_name) + '/'
    filename = folder + str(model_name) + '_' + str(width) + 'x' + str(height) + "_Normalized_TP.pdf"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    draw_bar_chart(diag,coco_labels,title,filename)

def draw_f1_score_graph(model_name, width, height):
    coco_labels = list(coco.values())
    f1_scores = []
    for val in coco_labels:
        f1_scores.append(get_F1_score(val))
    title = "F1 Scores"
    folder = str(model_name) + '/'
    filename = folder + str(model_name) + '_' + str(width) + 'x' + str(height) + "_F1_scores.pdf"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    draw_bar_chart(f1_scores, coco_labels, title, filename)

def draw_recall_graph(model_name, width, height):
    coco_labels = list(coco.values())
    recalls = []
    for val in coco_labels:
        recalls.append(get_recall(val))
    title = "Recalls"
    folder = str(model_name) + '/'
    filename = folder + str(model_name) + '_' + str(width) + 'x' + str(height) + "_recalls.pdf"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    draw_bar_chart(recalls, coco_labels, title, filename)

def draw_precision_graph(model_name, width, height):
    coco_labels = list(coco.values())
    precisions = []
    for val in coco_labels:
        precisions.append(get_F1_score(val))
    title = "Precisions"
    folder = str(model_name) + '/'
    filename = folder + str(model_name) + '_' + str(width) + 'x' + str(height) + "_precisions.pdf"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    draw_bar_chart(precisions, coco_labels, title, filename)

# rows equal to coco label id -1
# same with columns
# rows -> original classes
# columns -> detected classes
# last column indicates not found

def draw_graphs(model_name, width, height):
    draw_normalized_TP_graph(model_name, width, height)
    draw_f1_score_graph(model_name, width, height)
    draw_recall_graph(model_name, width, height)
    draw_precision_graph(model_name, width, height)

def test_all_dataset(model_name, width=None, height=None):
    val_err = 0
    dataset_path = "OID/Dataset/train"
    directory_contents = os.listdir(dataset_path)
    global time_dict
    time_dict = {}
    directory_contents_ = [directory_contents[36],directory_contents[21],directory_contents[17]]
    for subdirectory in directory_contents_:
        images = os.listdir(dataset_path + "/" + subdirectory)
        print("subdirectory: " + subdirectory)
        inference_time_per_class = 0
        for image_ in images[:2]:
            if image_.endswith(".jpg"):
                print("processing image: " + str(image_))
                full_image_path = dataset_path + "/" + subdirectory + "/" + image_
                try:
                    results, image_name, inference_time = predict_single_image(full_image_path, width, height)
                    inference_time_per_class += inference_time
                except ValueError:
                    val_err += 1
                    continue
                perform_analysis(results, image_name)
        # inference_time_mean = inference_time_per_class/len(images)        
        inference_time_mean = inference_time_per_class/2
        time_dict[subdirectory.lower()] = inference_time_mean
    draw_graphs(model_name, width, height)
    return time_dict

def save_confusion_matrix(model_name, width=None, height=None):
    mat = np.matrix(confusion_matrix)
    folder = str(model_name)+ "/"
    name = folder + str(width) +'x' + str(height) + "_confusion_matrix_" + str(model_name) + '.txt'
    os.makedirs(os.path.dirname(name), exist_ok=True)
    with open(name,'wb') as f:
        for line in mat:
            np.savetxt(f, line, fmt='%.2f')
    return name

def save_inference_time(model_name, width=None, height=None):
    folder = str(model_name)+ "/"
    name = folder + str(width) +'x' + str(height) + "_inference_time_" + str(model_name) + '.txt'
    os.makedirs(os.path.dirname(name), exist_ok=True)
    with open(name,'w') as f:
        f.write(str(time_dict))
    return name

def test_resolution(model_name, width=None, height=None):
    resolution = str(width) +'x' + str(height)
    test_all_dataset(model_name, width, height)
    name = save_confusion_matrix(model_name, width, height)
    confusion_matrix_names.append(name)
    inference_time_name = save_inference_time(model_name, width, height)
    inference_time_names.append(inference_time_name)


# Trenowanie modelu dla różnych rozdzielczości

In [ ]:
resolutions = [(3840,2160),(2560,1440),(1920,1200),(1920,1080),(1680,1050),(1600,900),(1360,768),(1024,768),(800,600)]
confusion_matrix_names = []
inference_time_names = []
time_dict = {}

# # ssd+mobilenet
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_640x640/1")
# confusion_matrix = [[0]*4 for i in range(3)]
confusion_matrix = [[0]*91 for i in range(90)]
test_resolution('ssd+mobilenet_v2', 640, 640)

for resolution in resolutions:
    confusion_matrix = [[0]*91 for i in range(90)]
    test_resolution('ssd+mobilenet_v2',resolution[0],resolution[1])

# faster_rcnn+resnet
detector = hub.load("https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1")
# confusion_matrix = [[0]*4 for i in range(3)]
confusion_matrix = [[0]*91 for i in range(90)]
test_resolution('faster_rcnn+resnet', 640, 640)

for resolution in resolutions:
    confusion_matrix = [[0]*91 for i in range(90)]
    test_resolution('faster_rcnn+resnet',resolution[0],resolution[1])

# retinanet+resnet
detector = hub.load("https://tfhub.dev/tensorflow/retinanet/resnet50_v1_fpn_640x640/1")
# confusion_matrix = [[0]*4 for i in range(3)]
confusion_matrix = [[0]*91 for i in range(90)]
test_resolution('retinanet+resnet', 640, 640)

for resolution in resolutions:
    confusion_matrix = [[0]*91 for i in range(90)]
    test_resolution('retinanet+resnet',resolution[0],resolution[1])




























# Porównanie efektywności w funkcji rozdzielczości dla konkretnej klasy i modelu

In [ ]:
def open_matrix(filename):
    with open(filename, 'r') as f:
        l = [[int(float(num)) for num in line.split(' ')] for line in f]
    return l

coco_readable_to_index = {value:key for key, value in coco.items()}

def get_all_detections(confusion_matrix, class_name):
    return confusion_matrix[coco_readable_to_index[class_name]-1][:-1]

def get_TP(confusion_matrix, class_name):
    return confusion_matrix[coco_readable_to_index[class_name]-1][coco_readable_to_index[class_name]-1]

def get_FN(confusion_matrix, class_name):
    return confusion_matrix[coco_readable_to_index[class_name]-1][-1]

def get_FP(confusion_matrix, class_name):
    return sum(get_all_detections(confusion_matrix, class_name)) - get_TP(confusion_matrix, class_name)

def get_precision(confusion_matrix, class_name):
    try:
        return get_TP(confusion_matrix, class_name) / sum(get_all_detections(confusion_matrix, class_name))
    except ZeroDivisionError:
        return 0
        
def get_recall(confusion_matrix, class_name):
    try:
        return get_TP(confusion_matrix, class_name) / ((get_TP(confusion_matrix, class_name))+get_FN(confusion_matrix, class_name))
    except ZeroDivisionError:
        return 0

def get_F1_score(confusion_matrix, class_name):
    numerator = 2 * get_TP(confusion_matrix, class_name)
    denominator = numerator + get_FP(confusion_matrix, class_name) + get_FN(confusion_matrix, class_name)
    try:
        return numerator/denominator
    except ZeroDivisionError:
        return 0

def get_normalized(confusion_matrix, class_name):
    tmp = []
    for row in confusion_matrix:
        normalized_row = []
        for value in row:
            normalized_row.append(value/(sum(row)))
        tmp.append(normalized_row)
    return tmp[coco_readable_to_index[class_name]-1][coco_readable_to_index[class_name]-1]

def read_matrices(model_name):
    matrices = []
    files = os.listdir(model_name)
    for file in files:
        if 'confusion_matrix' in file:
            resolution = file.split("_")[0]
            if resolution.split("x")[0] == "640":
                resolution_int = 0
                resolution = "Original"
            else:
                resolution_int = int(resolution.split("x")[0]) * int(resolution.split("x")[1])
            matrices.append((resolution_int, resolution, open_matrix(model_name + "/" + file)))
    matrices.sort(key = lambda x: x[0])
    return matrices

def get_normalized_for_resolutions(matrices_with_res , class_name):
    normalized_with_res = []
    for resolution in matrices_with_res:
        normalized_with_res.append((resolution[0], resolution[1], get_normalized(resolution[2],class_name)))
    return normalized_with_res

def get_f1_for_resolutions(matrices_with_res , class_name):
    f1_with_res = []
    for resolution in matrices_with_res:
        f1_with_res.append((resolution[0], resolution[1], get_F1_score(resolution[2],class_name)))
    return f1_with_res

def get_precision_for_resolutions(matrices_with_res , class_name):
    precision_with_res = []
    for resolution in matrices_with_res:
        precision_with_res.append((resolution[0], resolution[1], get_precision(resolution[2],class_name)))
    return precision_with_res

def get_recall_for_resolutions(matrices_with_res , class_name):
    recall_with_res = []
    for resolution in matrices_with_res:
        recall_with_res.append((resolution[0], resolution[1], get_recall(resolution[2],class_name)))
    return recall_with_res

    
def draw(resolutions_with_values, name, model_names):
    print(resolutions_with_values)
    fig = plt.figure()
    plt.rcParams["figure.figsize"] = (20,7)
    model_names_cycle = itertools.cycle(model_names)
    for resolution_with_value in resolutions_with_values:
        resolution_with_value.pop(0)
        x_positions = np.arange(len(resolution_with_value))
        values = [i[2] for i in resolution_with_value]
        labels = [i[1] for i in resolution_with_value]
        plt.plot(x_positions, values, linestyle='dashed', linewidth = 3,
        marker='o', markersize=12, label = next(model_names_cycle))
    plt.xticks(x_positions, labels)
    plt.legend(loc='best', prop={'size': 20})
    plt.tick_params(axis="both", labelsize=18)
    plt.title(name, fontsize=20)
    plt.xlabel("resolution", fontsize=20)
    plt.ylabel("value", fontsize=20)
    return fig

def print_pdf_for_model(model_names):
    matrices_with_res = []
    for model_name in model_names:
        matrices_with_res.append(read_matrices(model_name))
    if len(model_names) > 1:
        f1_pp = PdfPages("f1_score_by_resolution_comparison.pdf")
        precision_pp = PdfPages("precision_by_resolution_comparison.pdf")
        recall_pp = PdfPages("recall_by_resolution_comparison.pdf")
        normalized_pp = PdfPages("normalized_score_by_resolution_comparison.pdf")
    else:
        f1_pp = PdfPages(model_names[0] + "_f1_score_by_resolution.pdf")
        precision_pp = PdfPages(model_names[0] + "_precision_by_resolution.pdf")
        recall_pp = PdfPages(model_names[0] + "_recall_by_resolution.pdf")
        normalized_pp = PdfPages(model_names[0] + "_normalized_score_by_resolution.pdf")
    f1, normalized, precision, recall = ([] for i in range(4))
    for class_name in coco.values():
        for matrix_with_res in matrices_with_res:
            f1.append(get_f1_for_resolutions(matrix_with_res, class_name))
            normalized.append(get_normalized_for_resolutions(matrix_with_res, class_name))
            precision.append(get_precision_for_resolutions(matrix_with_res, class_name))
            recall.append(get_recall_for_resolutions(matrix_with_res, class_name))
        print(f1)        
        f1_pp.savefig(draw(f1, class_name, model_names))
        normalized_pp.savefig(draw(normalized, class_name, model_names))
        precision_pp.savefig(draw(precision, class_name, model_names))
        recall_pp.savefig(draw(recall, class_name, model_names))
        f1.clear()
        normalized.clear()
        precision.clear()
        recall.clear()
    f1_pp.close()
    precision_pp.close()
    recall_pp.close()
    normalized_pp.close()

In [ ]:
print_pdf_for_model(["ssd+mobilenet_v2", "faster_rcnn+resnet", "retinanet+resnet"])

# Wizualizacja macierzy pomyłek dla konkretnej klasy, modelu i rozdzielczości

In [ ]:
def draw_confusion_matrix(filename):
    coco = get_coco_labels()
    names =[]
    for key, val in coco.items():
        names.append(val)

    array = open_matrix(filename)

    new_arr = [row[:-1] for row in array]
    new_arr = np.array(new_arr)
    idx =[0,12,76]
    print(new_arr[np.ix_(idx, idx)])
    new_arr = new_arr[np.ix_(idx, idx)]

    df_cm = pd.DataFrame(new_arr, names, names)
    fig = plt.figure(figsize=(9,9))
    sn.set(font_scale=2.4) # for label size
    sn.heatmap(df_cm, annot=True, fmt='g', annot_kws={"size": 30}) # font size

    pdf = PdfPages("retinanet+resnet_conf_matrix.pdf")
    pdf.savefig(fig)
    pdf.close()

    plt.show()

In [ ]:
# draw_confusion_matrix("ssd+mobilenet_v2/640x640_confusion_matrix_ssd+mobilenet_v2.txt")
# draw_confusion_matrix("faster_rcnn+resnet/640x640_confusion_matrix_faster_rcnn+resnet.txt")
draw_confusion_matrix("retinanet+resnet/640x640_confusion_matrix_retinanet+resnet.txt")

#Porównanie czasu uczenia w funkcji rozdzielczości dla konkretnej klasy i modelu

In [ ]:
def get_time_for_resolutions(times_with_res , class_name):
    time_with_res = []
    for resolution in times_with_res:
        print(resolution)
        time_with_res.append((resolution[0], resolution[1], resolution[2][class_name]))
    return time_with_res

def read_time(model_name):
    matrices = []
    files = os.listdir(model_name)
    for file in files:
        if 'inference_time' in file:
            resolution = file.split("_")[0]
            if resolution.split("x")[0] == "640":
                resolution_int = 0
                resolution = "Original"
            else:
                resolution_int = int(resolution.split("x")[0]) * int(resolution.split("x")[1])
            matrices.append((resolution_int, resolution, open_time(model_name + "/" + file)))
    matrices.sort(key = lambda x: x[0])
    return matrices

def open_time(filename):
    with open(filename, 'r') as f:
        l = eval(f.read())
    return l

def draw_time(times_with_resolutions, name, model_names):
    fig = plt.figure()
    plt.rcParams["figure.figsize"] = (20,7)
    model_names_cycle = itertools.cycle(model_names)
    for time_with_resolution in times_with_resolutions:
        time_with_resolution.pop(0)
        x_positions = np.arange(len(time_with_resolution))
        values = [i[2] for i in time_with_resolution]
        labels = [i[1] for i in time_with_resolution]
        plt.plot(x_positions, values, linestyle='dashed', linewidth = 3,
        marker='o', markersize=12, label = next(model_names_cycle))
    plt.xticks(x_positions, labels)
    plt.legend(loc='best', prop={'size': 20})
    plt.tick_params(axis="both", labelsize=18)
    plt.title(name, fontsize=20)
    plt.xlabel("resolution", fontsize=20)
    plt.ylabel("time", fontsize=20)
    return fig

def print_pdf_for_model_time(model_names):
    times_with_res = []
    for model_name in model_names:
        times_with_res.append(read_time(model_name))
    if len(model_names) > 1:
        time_pp = PdfPages("inference_time_by_resolution_comparison.pdf")
    else:
        time_pp = PdfPages(model_names[0] + "_inference_time_by_resolution.pdf")
    time = []
    for class_name in coco.values():
        for time_with_res in times_with_res:
            time.append(get_time_for_resolutions(time_with_res, class_name))
        time_pp.savefig(draw_time(time, class_name, model_names))
        time.clear()
    time_pp.close()

In [ ]:
print_pdf_for_model_time(["ssd+mobilenet_v2", "faster_rcnn+resnet", "retinanet+resnet"])